University of Michigan - ROB 201 Calculus for the Modern Engineer

---

# Julia HW08: Chapters 8.1-10.4

### Topics Covered
- Improper Integrans
  - Vertical Asymptotes
  - Horizontal Asymptotes
  - Applications to Probabiliyu
- Computing Solutions of ODEs
  - Numerically using DifferentialEquations.jl
  - Analytically using the Matrix Exponential for Linear ODEs
- Understanding Stability of Equilibrium
  - Complex Exponentials
  - Role of Eigenvalues and Eigenvectors

<br>

<p align="center">
  <img src="./data/HW08FuturisticClassRoom.png", width="70%">
</p>

### Important
Friendly Checks in this file may be less friendly than previous homeworks as they are embeded in Markdown cells rather than runable, Julia cells. Please feel free to modify this notebook how you see fit to test your solutions.

# Problem 1: Handling Improper Integrals with QuadGK

Run the next cell, and note the try-catch construction that allows ``quadgk`` to fail without crashing the cell. You may want to use this in other circumstances. It is quite useful.

In [ ]:
using QuadGK
using Plots, LaTeXStrings

# Define the functions and their integration limits
functions = [
    (f=x -> 1/sqrt(x), a=0, b=1, title="Integral 1: Vertical Asymptote"),
    (f=x -> 1/x^2, a=1, b=Inf, title="Integral 2: Horizontal Asymptote"),
    (f=x -> exp(-x)/sqrt(x), a=0, b=Inf, title="Integral 3: Combination of Vertical and Horizontal Asymptotes"),
    (f=x -> 1/sqrt(abs(x)), a=-1, b=1, title="Integral 4: Integration Around a Singular Point")
]

println("If a horizontal asymptote is non-zero, then an improper integral 
    over [a, Inf) is always unbounded.\n")
println(" ")


# Calculate integrals and prepare plots
results = []
for (i, func) in enumerate(functions)
    try
        # Numerical integration
        result, error = quadgk(func.f, func.a, isinf(func.b) ? func.a + 10 : func.b)
        push!(results, (result=result, error=error))
    catch e
        # Handle integration error
        println("Failed to integrate function f$(i): $e")
        push!(results, (result="Failed", error="N/A"))  # Store an indicator of failure
    end

    # Generate x values for plotting, considering the domain
    x_vals = isinf(func.b) ? range(func.a + (iszero(func.a) ? 1e-3 : 0), stop=10, length=100) :
             range(func.a + (iszero(func.a) ? 1e-3 : 0), stop=func.b, length=100)

    # Plot each function
    label_text = isinf(func.b) ? "over [$(func.a), 10]" : "over [$(func.a), $(func.b)]"
    plot(x_vals, func.f.(x_vals), label=L"f$(i)(x) $label_text", title=func.title, xlabel=L"x", ylabel=L"f(x)")
    display(plot!(legend=:right))
end

# Output the results
for (i, res) in enumerate(results)
    if res.result != "Failed"
        println("Integral f$(i) result: $(res.result) with estimated error: $(res.error)")
    else
        println("Integral f$(i) could not be computed. Integral may be unbounded.")
    end
end

println(" ")
println("If a horizontal asymptote is non-zero, then an improper integral 
    over [a, Inf) is always unbounded.\n")
println("If a horizontal asymptote approaches zero too slowly, such as at rate 1/x 
    or slower, then an improper integral over [a, Inf) will be unbounded.\n")



In [ ]:
# Answer the question with true or false

I_studied_the_code_in_the_above_cell = false # true

### BEGIN SOLUTION 
I_studied_the_code_in_the_above_cell = true
### END SOLUTION 


In [ ]:
#Grader Cell. Worth one point
### BEGIN HIDDEN TESTS
@assert I_studied_the_code_in_the_above_cell
### END HIDDEN TESTS

# Problem 2: Improper Integrals in Probability

<br>

<p align="center">
  <img src="./data/HW08ProbabilityConcepts.png", width="70%">
</p>


<br>

### What is a Probability Density?

In formal courses on Probability and Statistics, you'll explore different ways to model (AKA, mathematically represent) random quantities including: probability measures, distributions, and densities. This section highlights **probability densities**--functions $f: \mathbb{R} \to [0, \infty)$ that are non-negative, sum up (integrate) to one, and where the integral 
$$\int_a^b f(x) \, dx$$
signifies the **probability of a random quantity** falling within the range $[a, b]$, such as your exam score being between 86\% and 93\%. Some probability densities are defined on bounded sets and others on unbounded sets. Those defined on unbounded sets require improper integrals.

### Given Probability Density

$$f(x) = \begin{cases} \frac{0.8415}{1 + ln(x)^3 \cdot x^3} & 1 \le x < \infty \\\\
0 & \text{elsewhere} \end{cases}$$



## P.2A: Compute the Mean (aka, Expected Value) for the given Probability Density

For a continuous random variable $X$ with a probability density function $f(x)$, the expected value (mean) is defined as:

$$
     μ:= E(X) := \int_{-\infty}^{\infty} x f(x) \, dx
$$



In [ ]:
# Compute the mean and call your answer mu and NOT μ

# Here is the density for 1 <= x <= infinity

f(x) = 0.8415 / (1 + (log(x))^3 * x^3)
# mu = 

### BEGIN SOLUTION 
using QuadGK
gMean(x) = x*f(x)
mu, error = quadgk(gMean, 1, Inf)
### END SOLUTION 
using Plots, LaTeXStrings
plt = plot(f, 1.0, 6.0, lw=2, color=:blue, guidefont = 15, legend=false,
    xlabel=L"x", ylabel=L"f(x)", title="Probability Density")
plot!([mu, mu], [0.0, f(mu)], lw=2, color=:red)
annotate!([(mu + 0.1, f(mu), text("Mean Value", :left, 10))])

In [ ]:
# Friendly Check
if abs(cos(mu) - 0.419698488636) > 1e-3
    println("You likely have an error in the mean.")
else
    println("Looking promising!")
end

In [ ]:
#Grader Cell. Worth 1 Point
### BEGIN HIDDEN TESTS
@assert abs(mu - 1.1376832174544242) < 1e-3
### END HIDDEN TESTS

## P.2B:  Compute the variance and the standard deviation for the given probability density

The variance ($\sigma^2$) is defined as:

$$
\text{var}(X) = E[(X - \mu)^2] = \int_{-\infty}^{\infty} (x - \mu)^2 f(x) \, dx
$$

where $\mu = E(X)$ is the mean. The standard deviation, $\sigma$, is the square root of the variance. 

In [ ]:
# Call you answers var and sigma

# var = 

# sigma = 

### BEGIN SOLUTION 
using QuadGK
gMean(x) = x*f(x)
mu, error = quadgk(gMean, 1, Inf)
gVar(x) = (x-mu)^2*f(x)
@show var, error = quadgk(gVar, 1, Inf)
sigma = sqrt(var)
### END SOLUTION 


In [ ]:
# Friendly Check
if abs(sin(var) - 0.413994698987) > 1e-3
    println("You likely have an error in the variance and if that is wrong, 
    your standard deviation will be wrong as well.")
else
    println("Looking promising! On your own, make sure sigma^2 = var.")
end

In [ ]:
#Grader Cell. Worth 1 Point
### BEGIN HIDDEN TESTS
@assert abs(var - 0.426) < 1e-1
@assert abs(sigma - 0.65) < 1e-1
### END HIDDEN TESTS

## P.2C:  Compute the median for the given probability density

The **median** of a probability density function $f(x)$ is the value $m$ such that:

$$
\int_{-\infty}^{m} f(x) dx = 0.5 = \int_{m}^{\infty} f(x) dx .
$$

The value of the random variable is equally likely to be less than or equal to $m$ as greater than or equal to $m$. The median represents the middle value of a data set when arranged in order. It is less sensitive to outliers and skewed data than the mean, making it a more reliable indicator of central tendency in those situations.

**Note:** You will need to devise a strategy to estimate the median within an error bound of $\bm{+/- 0.1}$. 

**Hint:** Start with m = mu and iterate from there.

In [ ]:
# Call your answer m

# m = 

### BEGIN SOLUTION 
using QuadGK
@show m = 0.8135*mu
integral, error = quadgk(f, 1, m)
### END SOLUTION 

In [ ]:
#Grader Cell. Worth 1 Point
### BEGIN HIDDEN TESTS
@assert abs(m - 0.92) <= 1e-1
### END HIDDEN TESTS

# ODEs Describe Important Physical Phenomena (e.g. Bipedal Walking)

  - Building an ODE model requires knowledge of a given domain; here we highight robotics and biomechanics.
  - By solving the ODE model, we learn facts about the system being modeled that one cannot see directly from the ODE itself. 
  - The 3-link model (on the right) represets walking on stilts or a ballerina walking tip-toe. There is no torque being applied at the bottom of the stance "foot". The torque about the stance foot arises from the center of gravity of the model being ahead of the mid-point of the two feet. This is accomplished by the torso being commanded to lean forward.
  - If you have ever walked on stilts, you will understand this phenomenon. If you have never walked on stilts, you are now prepared to give it a try. Please start with short stilts!


<br>


<p align="center">
    
 <img src="./data/HW08_Atalante.png" width="18%" style="display: inline-block; vertical-align: middle; margin-right: 80px">
    
<img src="./data/HW08_3LinkWalkerFrom1999.png" width="30%" style="display: inline-block; vertical-align: middle;">
</p>


**Left Image:** Wandercraft's exoskeleton [Atalante](https://en.wandercraft.eu/) allows patients with paraplegia to walk again; it provides all of the power for moving the patient's legs and contains an algorithm for balancing the exo+human. The first controllers used in Atalante came from Michigan's Bipedal Robotics Lab. 

**Right Image:** The 3-link walker is one of the simplest walking models that captures the essentials of the human gait. Michigan's foray into bipedal locomotion in 1999 began with the study of the 3-link walker, and over the years, progressed to Cassie and collaborating with Wandercraft. Video documentation of the journey is availale at 
[Bipedal Robotics Lab](https://www.youtube.com/user/DynamicLegLocomotion).

---

The cells below simulate the 3-Link model of bipedal walking. You have access to all of the code in the file ``HW08_DynamicModelUtilities.jl``.

> The underlying theory in 2007 was advanced PhD level:  https://grizzle.robotics.umich.edu/files/Westervelt_biped_control_book_15_May_2007.pdf
  
> Thanks to Dr. Yukai Gong, since 2022 the control algorithms developed in the Bipedal Robotics Lab can be learned by any Robotics MS student; see https://arxiv.org/abs/2105.08170. Yukai made a huge breakthrough that resulted in the overall control design being much more robust AND easier to understand. That is impact!

In [ ]:
# Import useful packages
using DifferentialEquations, LinearAlgebra
using Plots 
gr()

include("HW08_DynamicModelUtilities.jl")

# set the initial state
x0 = [pi/16,  -pi/16, pi/6, 1.0, -1.0, 0.5]

# set the simulation time 
T = (0.0, 10.0)  # start at time 0.0s, end at time 10.0s

# set up the ODE problem
problem = ODEProblem(threeLinkDynamics, x0, T, 0.0)

# assemble the collision detection and jump map functions into a callback
cb = ContinuousCallback(condition, affect!);

# solve the ODE problem using the Runge Kutta Tsitouras 5/4 Integrator
bipedSol = solve(problem, Tsit5(), dt=0.001,  callback=cb);

println("\n\nThe only thing to observe here is that the trajectories are converging
to periodic functions. Each jump is the start of a new step. The steady-state 
motion of the robot is called a stable limit cycle.")
pSol = plot(bipedSol)

In [ ]:
using Plots
include("HW08_DynamicModelUtilities.jl")

# Define the new time vector with evenly spaced points
t_new = range(bipedSol.t[1], stop=bipedSol.t[end], length=15*ceil(Int64, bipedSol.t[end]))  # Adjust 'length' as needed

# Use broadcasting to get interpolated values for all new time points
states = bipedSol.(t_new)

# Extract individual components from states
th1 = getindex.(states, 1)  # Extract first component (th1) from each state
th2 = getindex.(states, 2)  # Extract second component (th2) from each state
th3 = getindex.(states, 3)  # Extract third component (th3) from each state

# Call an animation function with the model's parameters
g, r, L, m, Mh, Mt, th3Des = modelParameters3LinkWalker()
anim = animate_biped(th1, th2, th3, r, L)

println("\n  The biped is being simulated with the Julia package DifferentialEquations.jl, 
the same package you will use shortly!")
println("\n  In 2001, the proof of the correctness of the control law used in the biped model 
merited the Axelby Award for Best Paper in the IEEE Transactions on Automatic Control.
Today, the same material is being mastered by undergrads; see https://wamiogunbi.com/biped-bootcamp")

println("\nWhen knees are included, the gait is more realistic, of course, and avoids dragging the feet.")


gif(anim, "biped_animation.gif", fps=15)  # Save as GIF


## Tutorial: How to Use the Package `DifferentialEquations.jl`

### Preface

The Van der Pol oscillator is an asypototically stable oscillator with nonlinear damping. It evolves according to the second-order differential equation:

$$ \ddot{x} - \mu (1 - x^2) \dot{x} + x = 0 $$

where $ x $ is the position and $ \mu $ is a scalar parameter indicating the **nonlinearity and the strength of the damping**.

  - When $x>1$, the term $\mu (1 - x^2) \dot{x}$ removes energy from the system, causing the solution to contract and when $x<1$, the term $\mu (1 - x^2) \dot{x}$ adds energy to the system, causing the solution to expand.

  - The result of the continual expanding and contracting is that the solution covergers to a periodic trajectory.

  - The van der Pol ODE is interesting in the study of nonlinear dynamics and chaos theory and has applications in various fields such as biology, electrical engineering, and physics.

In the follwoing cells, you will learn how to simulate the Van der Pol oscillator, extract the solution, and analyze it through visualization.


In [ ]:
# Importing Necessary Packages

using DifferentialEquations, Plots


## Step 1: Defining the Van der Pol Differential Equation

To simulate the Van der Pol oscillator, we first need to define it as a system of first-order ordinary differential equations (ODEs). We convert the second-order equation into two first-order equations: 

1. $ \dot{x} = y $
2. $ \dot{y} = \mu (1 - x^2) y - x $

Let's define this in Julia.


In [ ]:
# Defining the ODE

function van_der_pol!(du, u, p, t)
    x, y = u
    mu = p
    du[1] = y
    du[2] = mu * (1 - x^2) * y - x
end


## Step 2: Set Initial Conditions and Parameters

- **Initial Conditions**: Choose initial values for $ x $ and $ y $. For instance, $ x(0) = 0.2 $ and $ y(0) = 0.0 $.
- **Parameter $ \mu $**: This parameter controls the nonlinearity and the damping. We'll start with $\mu = 0.5 $.

Now, let's set these values and define the time span for our simulation.


In [ ]:
# Initial Conditions

u0 = [0.2, 0.0]  # Initial condition [x(0), y(0)]
tspan = (0.0, 60.0)  # Time span from t=0 to t=60
params = 0.5  # mu parameter


## Step 3: Solve the Differential Equation

We use the `DifferentialEquations.jl` package to numerically solve this equation. 


In [ ]:
# Solving the ODE

prob = ODEProblem(van_der_pol!, u0, tspan, params)
sol = solve(prob, TRBDF2())  # TRBDF2 and Tsit5 are commonly used solvers for nonlinear ODEs


## Step 4: Visualize the Solution

We can plot the trajectories of $ x $ and $y$ over time to observe the behavior of the system. Additionally, a phase plot of $ x $ versus $ y $ provides insight into the system's dynamics.


In [ ]:
# Plotting the Solution

plt1 = plot(sol, xlabel="Time", ylabel="x and y", title="Van der Pol Oscillator", legend=false)

plt2 = plot(sol, idxs=(1,2), xlabel="x", ylabel="y", lw=2, color=:brown, title="Phase Plot", legend=false)

display(plt1)
display(plt2)


 ## Optional: How to Make Nicer Plots

In [ ]:
using Plots

# Assuming `sol` is your solution object from the ODE solver
# Define the new time vector with evenly spaced points
t_new = range(sol.t[1], stop=sol.t[end], length=15*ceil(Int64, sol.t[end]))  # Adjust 'length' as needed

# Use broadcasting to get interpolated values for all new time points
states = sol.(t_new)

# Extract x and y components from states
x = getindex.(states, 1)  # Extract first component (x) from each state
y = getindex.(states, 2)  # Extract second component (y) from each state

# Plotting the first variable 'x' over the new time vector
plt1 = plot(t_new, x, lw=2, color=:red, xlabel="Time", ylabel="x", title="Interpolated Solution of x(t) over Time")

plt2 = plot(t_new, y, lw=2, color=:blue, xlabel="Time", ylabel="y", title="Interpolated Solution of y(t) over Time")

plt3 = plot(x, y, lw=1.5, color=:black, legend=false, title="Phase Plot: x(t) vs y(t)", xlabel="x", ylabel="y")
      scatter!([u0[1]], [u0[2]], color=:black, markersize=10)
# Adding an arrow using quiver!
quiver!([u0[1]], [u0[2]], quiver=([x[15]-x[1]], [y[15]-y[1]]), color=:red, label="Direction of Change")


display(plt1)
display(plt2)
display(plt3)


While the Above Ends the Tutorial on the ``DifferentialEquations`` Package, We Show one more Property of the van der Pol Oscillator:

  - For $\mu>0$, the van der Pol Oscillator has a stable LIMIT CYCLE, similar to our model for bipedal walking

In [ ]:
u0 = [2.0, -2.0]  # Initial condition [x(0), y(0)]
prob = ODEProblem(van_der_pol!, u0, tspan, params)
sol= solve(prob, Tsit5())  # TRBDF2 and Tsit5 are commonly used solvers for nonlinear ODEs

# vector of new time points
t_new = range(sol.t[1], stop=sol.t[end], length=15*ceil(Int64, sol.t[end]))  # Adjust 'length' as needed

# Use broadcasting to get interpolated values for all new time points
states = sol.(t_new)

# Extract x and y components from states
x = getindex.(states, 1)  # Extract first component (x) from each state
y = getindex.(states, 2)  # Extract second component (y) from each state

# Make plots
plot!(plt3, x, y, lw=1.5, color=:blue, legend=false, title="Phase Plot: x(t) vs y(t)", xlabel="x", ylabel="y",
linestyle=:dashdot)
      scatter!([u0[1]], [u0[2]], color=:blue, markersize=10)
xLimitCyle = x[450:end]; yLimitCyle = y[450:end]
plot!(plt3, xLimitCyle, yLimitCyle, lw=3, color=:red, legend=false)
scatter!([u0[1]], [u0[2]], color=:blue, markersize=10)
# Adding an arrow using quiver!
quiver!([u0[1]], [u0[2]], quiver=([x[3]-x[1]], [y[3]-y[1]]), color=:red, label="Direction of Change")

println("The boxy ellipse in red is called a LIMIT CYCLE. For the van der Pol Oscillator,
solutions starting inside the limit cycle coverge to it without ever crossing to the
outside, and solutions that start outside of the limit cycle coverge to it without
ever crossing to the inside. Hence, if the oscillator is perturbed away from the
limit cycle, it quickly returns to it.")

println("\nYour PC contains a Quartz Oscillator that drives the clock generator, a circuit within 
the PC, that produces various frequencies needed by different parts of the system. This 
includes the CPU clock, memory bus clock, and clocks for various peripherals. The Quartz Oscillator 
behaves like a van der Pol Oscillator: if perturbed, it quickly converges back to the nominal path.\n")


display(plt3)

# P.3A: ODE Multiple Choice
Below are four multiple-choice questions designed to test understanding of using the `DifferentialEquations.jl` package in Julia for simulating ordinary differential equations (ODEs). These questions include one question on the Van der Pol oscillator.

---

**Q1:** Which is the correct way to define a range to then interpolate?
1) `new_time_vector = range(start_value, stop_value, length=length_value)`
2) `new_time_vector = range_vector(start_value, start_value + length_value)`
3) `interpolate(sol, new_time_points)`
4) `states = sol.(new_time_vector)`

**A)** 1 and 3 \
**B)** 1 and 4 \
**C)** 2 and 3 \
**D)** 2 and 4 

**Q2:** Which of the following statements are true?
1) `getstate.(state, n)` can be used to extract the nth component from the state. \
2) `TRBDF2` and `Tsit5` are commonly used solvers for differential equations. \
3) `solve(equation_name, solver)` is the correct notation to find a solution to an differential equation.

**A)** 1 \
**B)** 1 and 2 \
**C)** 2 and 3 \
**D)** 1, 2, and 3 


**Q3:** In the context of simulating the Van der Pol oscillator, which term correctly describes the role of the parameter $ \mu $? 

**A)** Mu was Van der Pol's first name and his middle name, leading his friends to call him Mu-mu.  
**B)** It controls the amplitude of the oscillations.  
**C)** It adjusts the frequency of the steady-state solution.  
**D)** It influences the nonlinearity and the damping strength of the oscillator.


**Q4:** What is the correct syntax to define an ODE problem in Julia using `DifferentialEquations.jl`?

**A)** `defineODE(function, start, end, initial_conditions)`  
**B)** `ODESetup(ODEfunction, start_time, end_time, init_cond)`  
**C)** `ODEProblem(function, initial_conditions, time_span, parameters)`  
**D)** `ProblemODE(ODEfunction, parameters, init_conditions, t_range)`  

### Instructions

Replace X with A, B, C, or D.

In [ ]:
# Your answers
Answer1 = "X"
Answer2 = "X"
Answer3 = "X"
Answer4 = "X"

### BEGIN SOLUTION 
Answer1 = "A"
Answer2 = "A"
Answer3 = "D"
Answer4 = "B";
### END SOLUTION 

In [ ]:
# FriendlyCheck
include("HW08_DynamicModelUtilities.jl")
CheckAnswers = Answer1*Answer2*Answer3*Answer4
num_correct = evaluate_answers(CheckAnswers)
println("You currently have $num_correct correct answers out of 4.")


In [ ]:
#Grader Cell. Worth 1/2 Point
### BEGIN HIDDEN TESTS
# Answer1 = "A"
# Answer2 = "A"
# Answer3 = "D"
# Answer4 = "B";

@assert Answer1 == "A"
### END HIDDEN TESTS

In [ ]:
#Grader Cell. Worth 1/2 Point
### BEGIN HIDDEN TESTS
# Answer1 = "A"
# Answer2 = "A"
# Answer3 = "D"
# Answer4 = "B";

@assert Answer2 == "A"
### END HIDDEN TESTS

In [ ]:
#Grader Cell. Worth 1/2 Point
### BEGIN HIDDEN TESTS
# Answer1 = "A"
# Answer2 = "A"
# Answer3 = "D"
# Answer4 = "B";

@assert Answer3 == "D"
### END HIDDEN TESTS

In [ ]:
#Grader Cell. Worth 1/2 Point
### BEGIN HIDDEN TESTS
# Answer1 = "A"
# Answer2 = "A"
# Answer3 = "D"
# Answer4 = "B";

@assert Answer4 == "B"
### END HIDDEN TESTS

# Problem 3: More Fun with ODEs
## P.3B: Use the `DifferentialEquations.jl` package to solve the ODE 

Solve based on the information given in the next cell when the ODEs initial conditions are 

```Julia
x0 = 7.0
y0 = 15.0
```

and you need to know the solution's values over

```Julia
tspan = (0.0, 5.0) 
```

> ### Friendly Check
> 
> If you use the solver:
> ```Julia
> # Solve the ODE problem
> sol = solve(prob, Tsit5())  # Using Tsit5 solver, a common choice for nonlinear problems
> ```
> The answer will be:
> ```Julia
> sol[:, end]
> 2-element Vector{Float64}:
>   3.7305123678595633
>  -0.7785974658864031
> ```


In [ ]:
# Importing Necessary Packages
using DifferentialEquations, Plots

# Your and my ODE 

function myODE!(du, u, p, t)
    x, y = u
    du[1] = y
    du[2] = -5*y - x
end

# Put your code here
# call the solution by the generic name, sol

### BEGIN SOLUTION 

# Set initial conditions
u0 = [7.0, 15.0]  # Initial values for x and y
tspan = (0.0, 5.0)  # Time span from t=0 to t=10

# Create an ODE problem
prob = ODEProblem(myODE!, u0, tspan)

# Solve the ODE problem
sol = solve(prob, Tsit5())  # Using Tsit5 solver, a common choice for nonlinear problems

### END SOLUTION 

# Friendly Check
sol[:, end]

In [ ]:
#Grader Cell. Worth 3 Points
### BEGIN HIDDEN TESTS
using LinearAlgebra
@assert norm(sol[:, end] - [3.7305123678595633; -0.7785974658864031]) < 5e-3
### END HIDDEN TESTS

## Analyzing the Solutions of ODEs

We’ve seen that solving ODEs numerically is not too difficult, while solving them analytically is often only possible in special cases. In practice, each field — Robotics, Aerospace Engineering, Chemical Engineering, Physics, Economics, etc. — develops its own models of systems using ODEs.  

A key **domain-independent skill** is determining the **local stability** of equilibrium points.

### Definitions

- An **equilibrium point** $x_e \in \mathbb{R}^n$ of the system $\dot{x} = f(x)$ is any point where $f(x_e) = 0_{n \times 1}$.

- **Local asymptotic stability (informal):** An equilibrium point $x_e$ is locally asymptotically stable if, when starting close enough to $x_e$, the system trajectories converge to $x_e$ as $t \to \infty$.

- **Local asymptotic stability (formal):** An equilibrium point $x_e$ is locally asymptotically stable if there exists some $\delta > 0$ such that, for all initial conditions $x_0$ with $\|x_0 - x_e\| < \delta$, we have $\lim_{t \to \infty} x(t) = x_e$.

### Illustration

Consider the **Van der Pol Oscillator run backward in time**.  
For this system, trajectories with initial conditions inside the limit cycle converge to the origin.  

This shows that the neighborhood size $\delta$ in the definition does not have to be extremely small. The definition only guarantees that $\delta$ is some positive number, but in practice it can often be fairly large.

---

**Note:** The next example cell uses `xLimitCycle` and `yLimitCycle` from the Van der Pol demonstration.


In [ ]:
function backwardVanDerPol!(du, u, p, t)
    x, y = u
    # backward van der Pol: origin is now attractive
    mu = p
    du[1] = -y
    du[2] = -(mu * (1 - x^2) * y - x)
end

# Set initial conditions
u0 = [1.1, 1.7]  # Initial values for x and y
tspan = (0.0, 30.0)  # Time span from t=0 to t=10
params = 0.5  # mu parameter

# Create an ODE problem
prob = ODEProblem(backwardVanDerPol!, u0, tspan, params)

# Solve the ODE problem
sol = solve(prob, Tsit5())  # Using Tsit5 solver, a common choice for nonlinear problems

t_new = range(sol.t[1], stop=sol.t[end], length=15*ceil(Int64, sol.t[end]))  # Adjust 'length' as needed

# Use broadcasting to get interpolated values for all new time points
states = sol.(t_new)

# Extract x and y components from states
x = getindex.(states, 1)  # Extract first component (x) from each state
y = getindex.(states, 2)  # Extract second component (y) from each state

plt4 = plot(xLimitCyle, yLimitCyle, lw=3, color=:red, legend=false)
plot!(x, y, lw=1.5, color=:black, legend=false, title="Phase Plot: x(t) vs y(t)", xlabel="x", ylabel="y")
      scatter!([u0[1]], [u0[2]], color=:black, markersize=10)

# Adding an arrow using quiver!
quiver!([u0[1]], [u0[2]], quiver=([x[3]-x[1]], [y[3]-y[1]]), color=:red, label="Direction of Change")

println("\nWhy does this version of van der Pol converge to the origin, an equilibirum point, whereas 
the previous version spirals away from the equilibirum point? Inquiring minds want to know! \n")


display(plt4)

# Problem 4: Eigenvalues, Eigenvectors, and the Matrix Exponential 

<br>

<p align="center">
  <img src="./data/HW08ExponentialStabilityLinearSystems.png", width="90%">
</p>

**Note:** Later in the HW, we'll provide code that allows you to explore why the above result is true. Next, we tie linear ODEs to nonlinear ODES through their (Jacobian) linearization.

<br>


<p align="center">
  <img src="./data/HW08LinearizationAboutEquilibriumPoint.png", width="90%">
</p>

### Important

Because $x_e + \delta x(t)$ is an approximate solution to the nonlinear model, if the Jacobian matrix of the linearization $A:=\frac{\partial f(x_e)}{\partial x} $ has all eigenvalues with negative real parts, then $x_e + \delta x(t)$ will converge to $x_e$ as $t$ tends to infinity.

## P.4A: Compute the Eigenvalues of the Jacobian Linearization of the Van der Pol Oscillator

We consider the linearized system $\delta \dot{x} = A \, \delta x,$ where $A := \frac{\partial f(x_e)}{\partial x}, \qquad x_e = \begin{bmatrix} 0 \\ 0 \end{bmatrix}.$

Even though this problem is set up for you to use **ForwardDiff** to compute the Jacobian, you may use another method if you prefer.  

In the end, you must ensure that:

- $A$ is a $2 \times 2$ real-valued matrix, and  
- You correctly compute both $A$ and its eigenvalues.


In [ ]:
using ForwardDiff, LinearAlgebra

# right-hand side of the van der Pol oscillator
# Define the function f(x, y) returning a vector
function f(z)
    x, y = z
    return [y, 0.5 * (1 - x^2) * y - x]
end

## Compute the Jacobian matrix at the origin
# A = ??

## Compute the eigenvalues of the Jacobian matrix
# eigenvalues = ??

### BEGIN SOLUTION 

# Compute the Jacobian matrix at the origin
xe = [0.0; 0.0]
A = ForwardDiff.jacobian(f, xe)

# Compute the eigenvalues of the Jacobian matrix
E = eigen(A)
eigenvalues = E.values

### END SOLUTION

In [ ]:
# friendly check
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong
#
RealParts = real.(eigenvalues)
ImagParts = imag.(eigenvalues)

# Run Test
is_it_correct_check1 = abs( abs(RealParts[1]*ImagParts[2]) - 0.24206145913796354 ) < 1e-4 ? "Yes" : "No"
@show is_it_correct_check1;

In [ ]:
# Grader Cell. Worth 1 Point
### BEGIN HIDDEN TESTS
RealParts = real.(eigenvalues)
@assert norm(A - [0.0 1.0; -1.0 0.5]) < 1e-2
@assert abs(RealParts[1] - 0.25) < 1e-2
###END HIDDEN TESTS

## P.4B: Compute the Eigenvalues of the Jacobian Linearization of the "Backward-Running" Van der Pol Oscillator

We consider the linearized system $\delta \dot{x} = A \, \delta x,$ where $A := \frac{\partial f(x_e)}{\partial x}, \qquad x_e = \begin{bmatrix} 0 \\ 0 \end{bmatrix}.$

As before, this problem is set up for you to use **ForwardDiff** to compute the Jacobian, but you may use other methods if you prefer.  

In the end, you must ensure that:

- $A$ is a $2 \times 2$ real-valued matrix, and  
- You correctly compute both $A$ and its eigenvalues.

**Hint:** The eigenvalues of the *backward-running* Van der Pol oscillator will be the **negatives** of the eigenvalues of the *forward-running* Van der Pol oscillator.


In [ ]:
using ForwardDiff, LinearAlgebra

# right-hand side of the backward van der Pol oscillator
# Define the function f(x, y) returning a vector
function f(z)
    x, y = z
    return [y, 0.5 * (1 - x^2) * y - x]
end


## Compute the Jacobian matrix at the origin
# A = ??

## Compute the eigenvalues of the Jacobian matrix
# eigenvalues = ??

### BEGIN SOLUTION 

# Compute the Jacobian matrix at the origin
xe = [0.0; 0.0]
A = ForwardDiff.jacobian(f, xe)

# Compute the eigenvalues of the Jacobian matrix
E = eigen(A)
eigenvalues = E.values

### END SOLUTION

# We are checking both A and eigenvalues

In [ ]:
# Grader Cell. Worth 1 Point
### BEGIN HIDDEN TESTS
RealParts = real.(eigenvalues)
@assert norm(A) - 1.5 < 1e-2
@assert abs(sum(RealParts) - 0.5) < 1e-2
###END HIDDEN TESTS

# Understanding Eigenvalues 

How and why do eigenvalues determine whether initial conditions near an equilibrium lead to solutions that coverge to the equlibrium or diverge from it? 

### Key Topics
- Complex Exponentials
- Matrix Exponentials
- $e^{A \, t} \cdot v = e^{\lambda t} \cdot v$ when $A \cdot v = \lambda \cdot v$:
  * Hence, if $v$ and $\lambda$ are real, then $ e^{\lambda t} \cdot v$ is a vector of real exponentials and easy to understand.
  * However, if $v$ and $\lambda$ are complex, then $ e^{\lambda t} \cdot v$ is a vector of complex exponentials, so what does that mean, exactly? 


## Optional: Primer on Complex Numbers

If you are good with complex numbers, you can skip this.

<p align="center">
  <img src="./data/HW08ReviewComplexNumbers.png", width="95%">
</p>

<br>

Here's a mini-tutorial on complex numbers in Julia that covers the basics, including creating a complex number, performing arithmetic operations, extracting real and complex parts, and more:

#### 1. Creating a Complex Number
In Julia, complex numbers are created using the `Complex` type. A complex number $z$ is represented as $a + bi$, where $a$ is the real part and $b$ is the imaginary part.

```julia
z = 3 + 4im  # 'im' denotes the imaginary unit in Julia
```

#### 2. Adding and Multiplying
You can perform arithmetic operations like addition, subtraction, multiplication, and division on complex numbers just like on real numbers.

```julia
z1 = 3 + 4im
z2 = 1 + 2im

# Addition
addition = z1 + z2  # Outputs 4 + 6im

# Multiplication
multiplication = z1 * z2  # Outputs -5 + 10im
```

#### 3. Real and Imaginary Parts
To extract the real or imaginary parts of a complex number, use the `real()` and `imag()` functions, respectively.

```julia
real_part = real(z1)  # Outputs 3
imaginary_part = imag(z1)  # Outputs 4
```

#### 4. Complex Conjugate
The complex conjugate of a complex number $a + bi$ is $a - bi$. You can get the conjugate in Julia using the `conj()` function.

```julia
conjugate = conj(z1)  # Outputs 3 - 4im
```

#### 5. Magnitude and Angle
The magnitude (or absolute value) of a complex number can be found using `abs()`, and the angle (or phase) can be obtained with `angle()`.

```julia
magnitude = abs(z1)  # Outputs 5
angle = angle(z1)  # Outputs approximately 0.927 radians
```
If you recall **polar coordinates** in the plane, then you understand that you can represent a pair of numbers (x, y) as ($\rho$, $\theta$), where $\rho$ is the magnitude, $\sqrt{x^2 + y^2}$, and $\theta$ is the phase, ``atan2(x,y)``. The same is true for a complex number $z:=x + i y$ which can be though of as a pair $(x, y)$ where $x$ is the real part of $z$ and $y$ is the imaginary part.

#### Additional Operations
Julia also supports other complex number functions like `abs2()` for the square of the absolute value and `cis(theta)` to create a complex number given an angle $\theta$ ( $e^{i\theta}$ ).

```julia
# Square of the absolute value
square_magnitude = abs2(z1)  # Outputs 25

# Create complex number from an angle
theta = pi / 4
complex_from_angle = cis(theta)  # Outputs approximately 0.707 + 0.707im
```


In [ ]:
# Use this cell to practice with complex numbers. Nothing here is graded.
#
@show im^2 #  remains a complex TYPE with zero imaginary part
@show real(im^2)  # becomes a real number
@show imag(im^2)  # becomes a real number
@show z = -3 + 4im # im = sqrt(-1)
@show rho = abs(z)  # magnitude, similar to polar coordinates
@show theta = angle(z)  # angle, similar to polar coordinates
display([z;   rho*exp(im*theta)]); # z = |z|*exp(im * angle(z))

# Add your commands

# Problem 5: Complex Exponentials and the Matrix Exponential

<p align="center">
  <img src="./data/HW08ComplexExponentials.png", width="90%">
</p>

Study the code and play with the values of a and w in the following cell. Use your observatons to answer the next question.

In [ ]:
using Plots, LinearAlgebra

# Define constants a and w
a = 0.25  # Real coefficient
w = 2.0  # Imaginary coefficient

# Define the time vector t from 0 to 10 in steps of 0.01
t = 0:0.01:10

# Calculate the complex exponential e^(z*t) 
z = a + im*w  # Define the complex number z = a + iw
exp_zt = exp.(z .* t)  # Element-wise exponentiation

# Compute the magnitude
magnitude_exp_zt = norm.( exp.(z .* t) )

# Extract real and imaginary parts
real_part = real.(exp_zt)
imaginary_part = imag.(exp_zt)

# Plotting: We must plot the real and imaginary parts separately.
# Plots does not know what to do with complex numbers
plt1 = plot(t, 0*t, label="", lw=2, color=:gold, title="Understanding exp((a + iw)t)")
plot!(t, real_part, label="Real part", color=:blue, lw=2)
plot!(t, imaginary_part, color=:red, lw=2, label="Imaginary part")
plot!(t, magnitude_exp_zt, label="Magnitude = exp(at)", color=:black, lw=2, linestyle=:dash)
plot!(t, -magnitude_exp_zt, label="Negative of Magnitude", color=:gray, lw=2, linestyle=:dash)

xlabel!("Time (t)")
ylabel!("Value")

# Display the plot
display(plt1)

# Compute exp(at) for all t
exp_at = exp.(a .* t)

# Compute the required functions
exp_at_cos_wt = exp_at .* cos.(w .* t)
exp_at_sin_wt = exp_at .* sin.(w .* t)

# Plotting the components and exp(at)
plt2 = plot(t, exp_at_cos_wt, label="exp(at)*cos(wt)", color=:blue, lw=2, title="Component Functions of exp((a + iw)t)")
plot!(t, exp_at_sin_wt, label="exp(at)*sin(wt)", color=:red, lw=2)
plot!(t, exp_at, label="exp(at)", color=:black, lw=2, linestyle=:dash)
plot!(t, -exp_at, label="-exp(at)", color=:gray, lw=2, linestyle=:dash)

xlabel!("Time (t)")
ylabel!("Value")

# Display the plot
display(plt2)


<!-- 
<p align="center">
  <img src="HW08ComplexExponentialsProperties.png", width="90%">
</p>
 -->

## P.5A: Multiple choice questions on complex exponential functions

These questions cover various scenarios involving the parameters $a$ and $w$ in the complex exponential, $ e^{(a + i w) t},$
to enhance your understanding of how the real and imaginary parts of the complex exponential function behave under different conditions.

**Q1:** What can we control by adjusting the value of a in $e^{(a + iw)t}$?

A) The oscilation frequency of the function.  
B) The rate of exponential growth or decay.  
C) The initial phase shift.  
D) The chance that you will get an A in this class.

**Q2:** What is the behavior of the real and imaginary parts of $e^{(a + iw)t}$ as $t$ approaches infinity if $a < 0$?

A) Both parts approach zero.  
B) The real part approaches zero but the imaginary part approaches infinity.  
C) Both parts approach infinity.  
D) The real part approaches infinity but the imaginary part approaches zero.

**Q3:** If $w \neq 0$ and the limit as $t$ approaches infinity of the real part of $e^{(a + iw)t}$ goes to zero, what happens to the imaginary part?

A) The imaginary part can do whatever it wants.  
B) The imaginary part also goes to zero. 
C) The imaginary part remains constant.  
D) The imaginary part converges to plus infinity or minus infinity. 

**Q4:** When $a = 0$ and $w \neq 0$, what is the nature of $e^{(a + iw)t}$?

A) It is purely real for all $t$.  
B) It is purely imaginary for all $t$.  
C) Its real and imaginary parts oscillate with a magnitude of 1.   
D) It remains constant with a magnitude of 1. 


In [ ]:
# Replace X with A, B, C, or D. Do not remove the double quote marks

Answer1 = "X"
Answer2 = "X"
Answer3 = "X"
Answer4 = "X"

### BEGIN SOLUTION 

Answer1 = "B"
Answer2 = "A"
Answer3 = "B"
Answer4 = "C";

### END SOLUTION


In [ ]:
# FriendlyCheck
include("HW08_DynamicModelUtilities.jl")
CheckAnswers = Answer1*Answer2*Answer3*Answer4
num_correct = evaluate_answers02(CheckAnswers)

println("You currently have $num_correct correct answers out of 4. 
    They are worth 0.5 points each.")

In [ ]:
#Grader Cell. 0.5 points
### BEGIN HIDDEN TESTS
# Answer1 = "C"
# Answer2 = "A"
# Answer3 = "D"
# Answer4 = "D";
@assert (Answer1=="B")
### END HIDDEN TESTS

In [ ]:
#Grader Cell. 0.5 points
### BEGIN HIDDEN TESTS
@assert (Answer2=="A")
### END HIDDEN TESTS

In [ ]:
#Grader Cell. 0.5 points
### BEGIN HIDDEN TESTS
@assert (Answer3=="B")
### END HIDDEN TESTS

In [ ]:
#Grader Cell. 0.5 points
### BEGIN HIDDEN TESTS
@assert (Answer4=="C")
### END HIDDEN TESTS

<p align="center">
  <img src="./data/HW08MatrixExponentialDefinition.png", width="90%">
</p>
<br>

<br>
<p align="center">
  <img src="./data/HW08Solution2LinearODEs.png", width="90%">
</p>
<br>

To make things concrete, consider:

<br>
<p align="center">
  <img src="./data/HW08MassSpringDamper.png", width="40%">
</p>
<br>
the classic mass-spring-damper. If you were to rotate the image with the mass on top and the wall being the ground, the system would become a simplified version of a shock absorber in a car (after adding in gravity). The spring compresses to absorb shocks, but on its own, it would continue to oscillate. The role of the damper is to remove energy from the system (similar to friction), causing the oscillations to die out. As derived in the textbook, the differential equation model is

$$\left[\begin{array}{c} \dot{x}_{1}  \\\\ \dot{x}_2 \end{array} \right] = \left[\begin{array}{cc} 0 & 1 \\\\ - \frac{k}{m} & -\frac{b}{m} \end{array}\right] \left[ \begin{array}{c} x_1  \\\\ x_2 \end{array}\right],$$

where $x_1$ is the position of the mass (relative to the rest position of the spring and the wall) and $x_2$ is the velocity of the mass. The parameters are $m$ the mass of the vehicle, $k$ the spring constant, and $b$ the damping constant.

The initial conditions for the ODE come from the initial position of the mass (relative to the rest position of the spring and the wall) and the initial velocity of the mass.

---

A numerical solution is given first using ``DifferentialEquations.jl``:

```julia
# Nominal parameters
m = 7.0   # Mass in kg
k = 2.0   # Spring constant in N/m
b = 4.0   # Damping coefficient in Ns/m
```

In [ ]:
# Model and simulate the mass spring damper system
# dx/dt = A*x(t)

using DifferentialEquations, Plots

# Parameters
m = 7.0   # Mass in kg
k = 2.0   # Spring constant in N/m
b = 4.0   # Damping coefficient in Ns/m

# Package the parameters for passing them to DifferentialEquations
params = (m, k, b)  # m, k, b

# Differential equation du/dt = A*u
# You can use x, but u is the default notation with ODEProblem.
function mass_spring_damper!(du, u, p, t)
    m, k, b = p  # unpack parameters
    # Various methods to formulate the ODE
    if false # first two methods work
        du[1] = u[2]
        du[2] = -(k/m) * u[1] - (b/m) * u[2]
    elseif true
        A = [0 1; -k/m -b/m]
        temp = A * u  # Calculate the product of A and u
        du[1] = temp[1]  # Assign the first component to du[1]
        du[2] = temp[2]  # Assign the second component to du[2]
    else # This method does not work with ODEProblem
        # You can ask your favorite LLM why.
        A = [0 1; -k/m -b/m]
        du = A*du
    end
end


# Initial conditions: assuming initial displacement of 1 meter and initial velocity of 0 m/s
u0 = [1.0, 0.0]

# Time span for the simulation
tspan = (0.0, 20.0)

# Solve the ODE
prob = ODEProblem(mass_spring_damper!, u0, tspan, params) # (odeModel, initialConditions, timespan, parameters)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)


A=[0 1; -k/m -b/m]
println("In the next cell, we'll relate the response to the eigenvalues of the matrix A, $A")

display(A)

# Plot the solution. Several different ways to make the plot are illustrated
# Find one you like and stick with it.
if true
    # Plot the results
    plt1 = plot(sol.t, [x[1] for x in sol.u], label="Displacement (m)", xlabel="Time (s)", ylabel="Displacement (m)", 
        title="Mass-Spring-Damper Initial Condition Response")
    plot!(sol.t, [x[2] for x in sol.u], label="Velocity (m/s)", ylabel="Velocity (m/s)", legend=:topright)
elseif true
    plt1 = plot(sol, xlabel="Time (s)", ylabel="State", title="Mass-Spring-Damper System Response", legend=:topright)
else
    plt1 = plot(sol, xlabel="Time (s)", ylabel="State", title="Mass-Spring-Damper System Response", legend=:topright)
    plot!(sol.t, [x[1] for x in sol.u], label="Displacement (m)")
    plot!(sol.t, [x[2] for x in sol.u], label="Velocity (m/s)")
end

display(plt1)


In [ ]:
using LinearAlgebra

# Mass spring damper default values. 
# Change b in the previous cell and run it for b = 3, 1 and 10
# Then run this cell and see that we get the same results.

A = [0 1; -k/m -b/m]
println("Here is the matrix A:")
display(A)
F = eigen(A)
println("Here are the eigenvalues of matrix A:")
display(F.values)
if norm(imag.(F.values)) < 1e-6
    println("Note they are real valued, which yields monontonic trajectories")
else
    println("Note they are complex valued, which yields oscillations")
end
println("\nHere is the matrix exp(A):")
expA = exp(A)
display(expA)
G = eigen(expA)
println("Here are the eigenvalues of matrix exp(A):")
display(G.values)
println("They are equal to the exponential of the eigenvalues of matrix A:")
expEigenValuesA = exp.(F.values)
display(expEigenValuesA)

# data from the previous cell
t = range(tspan[1], tspan[2], 1000)
x0 = u0 
traj = x0
for k = 1:length(t)-1
 traj = [traj exp(A*t[k])*x0]
end

println("\nSolving dx/dx = Ax, x(t0)=x0 via x(t) = exp(A*t)*x0. Because we know the exact solution
to the ODE, we can analyze its properties to understand when it oscillates while 
exponentially decaying, when it is decays monotonically, and when it blows up.\n\n")

plt2 = plot(t, traj[1,:], xlabel="Time (s)", ylabel="State", title="Mass-Spring-Damper exp(At)*x0", 
    legend=:topright, label="Displacement (m)")
    plot!(t, traj[2,:], label="Velocity (m/s)")
display(plt2)
println("Go back two cells, change b and the then re-run these last two cells. Focus on
whether the eigenvalues are real or complex. If you set the damping  b to a negative number
you can make the solution explode.")

## What Do Eigenvectors Do?

Eigenvectors define subspaces for initial conditions $x_0$ that create solutions that are vector versions of a single complex exponential, $e^{\lambda t} \cdot x_0$.

**Fundamental Relation:** $A v = \lambda v$

#### Case 1: $\lambda$ is a Real Eigenvalue of $A$

- For any $x_0 \in \text{span}\{v\}$, $x(t) = e^{\lambda t} \, x_0$
- Behavior depends on $\lambda$:
  - Increasing exponentially if $\lambda > 0$
  - Decreasing exponentially if $\lambda < 0$
  - Constant if $\lambda = 0$

#### Case 2: $\lambda$ is a Complex Eigenvalue of $A$

- $\lambda = \lambda_R + i \lambda_I$
- $v = v_R + i v_I$
- For any $x_0 \in \text{span}\{v_R, v_I\}$, the response involves complex exponentials (see Proposition 9.54).  

<br>
<p align="center">
  <img src="./data/HW08MatrixExponentialEigenvectors.png" width="90%">
</p>
<br>

**Note:** Everything in equation (9.68) is a **real vector-valued function of time**. This can be verified by plotting $e^{At} v_R$ and $e^{At} v_I$.


In [ ]:
using LinearAlgebra, Plots


println("Here are the eigenvalues and eigenvectors of A")
display(F)


# Correctly handling trajectory calculations and plotting
if norm(imag.(F.values)) < 1e-6
    println("\n For real eigenvectors, we use thenm directly as initial conditions 
    for dx/dt = Ax, x(t0) = x0\n\n")
    for k = 1:size(F.vectors, 2)
        x0 = real(F.vectors[:, k])
        println("\nx0 is chosen as v$k, x0 = $x0")
        
        traj = [exp(A*t[i])*x0 for i in eachindex(t)]  # Efficient matrix computation for each time point
        traj = reduce(hcat, traj)  # Concatenating column-wise correctly
        
        plt = plot(t, traj', label=["Displacement (m)" "Velocity (m/s)"], legend=:topright, 
            xlabel="Time (s)", ylabel="State", guidefont=15)
        display(plt)
    end
else
    println("\n For complex eigenvectors, we use their real and imaginary parts 
    as initial conditions for dx/dt = Ax, x(t0) = x0\n\n")
    for i = 1:size(F.vectors, 2)
        if i == 1
            x0 = real(F.vectors[:, i])
        else
            x0 = imag(F.vectors[:, i])
        end
        println("\nx0 = $x0")
        
        traj = [exp(A*t[i])*x0 for i in eachindex(t)]  # Efficient matrix computation for each time point
        traj = reduce(hcat, traj)  # Concatenating column-wise correctly
        
        plt = plot(t, traj', label=["Displacement (m)" "Velocity (m/s)"], legend=:topright, 
            xlabel="Time (s)", ylabel="State", guidefont=15)
        display(plt)
    end
end


Relating Solutions of $\dot{x} = Ax$, $x(0)=x_0$ to Eigenvalues and Eigenvectors of $A$.



In [ ]:
# Run Me. Do NOT change me.

using LinearAlgebra

A = [-1.20765   -0.576936  -4.65319  -1.30087
 -5.11041    0.866501  -9.48172  -4.64823
  0.841043  -0.389652  -1.39981   0.214517
 -1.12432    1.35583    4.62568  -0.259039]

F = eigen(A)


println("\n Eigenvalues Follow \n")
display(F.values)

println("\n Eigenvectors Follow, 1 through 4 \n")

for i = 1:size(A,1)
    display(F.vectors[:,i])
end




## P.5B: Give an initial condition $x_0 \in \mathbb{R}^4$ such that the norm of the solution $e^{A t} \cdot x_0$ converges to zero without oscillations

### Notes
- Norm of solution is $||e^{A t} \cdot x_0|| $.
- While your answers should be real vectors, if their imaginary part is zero (to numerical precision), our grading will still count you as correct. If you REALLY want to make sure your answer is a real vector, use the `real.()` and `imag.()` commands in Julia. It's totally optional. 
- If you are not using the eigenvectorrs, or parts of them in your solutions, then you are missing a crucial part of the problem.


In [ ]:
# x0 = 

### BEGIN SOLUTION 

x0 = F.vectors[:, 1]

if false
    # Here is  how JWG built the A matrix
    Lambda = diagm([1, -1, -1+im, -1-im])
    using Random
    Random.seed!(123456)
    M = randn(4,4)
    display(M)
    @show cond(M)
    v1 = M[:,1]
    v2 = M[:,2]
    v3 = M[:,3] + im*M[:,4]
    v4 = M[:,3] - im*M[:,4]
    V = [v1 v2 v3 v4]
    A = V * Lambda * inv(V)
    display(A)
    A = real.(A)
end

### END SOLUTION


In [ ]:
# Second way to do your own friendly check
using Plots
t = 0:0.01:5  # no need to collect, range itself can be operated on directly

# Calculate norm of solutions using broadcasting and matrix operations
norm_sol = [norm(exp(A * tk) * real.(x0)) for tk in t]

plot(t, norm_sol, label="Norm of solution to dx/dt = Ax, x(0)=x0", xlabel="t", ylabel="||exp(A*t) x0||")


In [ ]:
#Grader Cell. 1 point
### BEGIN HIDDEN TESTS
@assert rank([x0 F.vectors[:, 1]], atol=1e-3) == 1
### END HIDDEN TESTS


## P.5C: Give an initial condition $x_0 \in \mathbb{R}^4$ such that the norm of the solution $e^{A t} \cdot x_0$ converges to zero with "oscillations" (not monotonic)

**Note:** The norm can never be less than zero, so the term "oscillations" is at best approximate here, as illustrated in the image below. 

<br> 
<p align="center">
  <img src="./data/HW08NormSolutionDecayingExponentialWithSinusoid.png", width="60%">
</p>
<br>


In [ ]:
# x0 = 

### BEGIN SOLUTION 

x0 = real.(F.vectors[:,2])

# using Plots
# t = 0:0.01:5  # no need to collect, range itself can be operated on directly

# # Calculate norm of solutions using broadcasting and matrix operations
# norm_sol = [norm(exp(A * tk) * real.(x0)) for tk in t]

# plot(t, norm_sol, label="Norm of solution to dx/dt = Ax, x(0)=x0", xlabel="t", ylabel="||exp(A*t) x0||")

### END SOLUTION




In [ ]:
# Do your own checking here



In [ ]:
#Grader Cell. 1 point
### BEGIN HIDDEN TESTS
@assert norm(imag.(x0))<1e-4
@assert rank([x0 F.vectors[:, 2:3]],  atol=1e-3) == 3
### END HIDDEN TESTS

## P.5D: Give an initial condition $x_0 \in \mathbb{R}^4$ such that the norm of the solution $e^{A t} \cdot x_0$ converges to infinity with NO oscillations 

In [ ]:
# x0 = 

### BEGIN SOLUTION 

x0 = real.(F.vectors[:,4])

# using Plots
# t = 0:0.01:5  # no need to collect, range itself can be operated on directly

# # Calculate norm of solutions using broadcasting and matrix operations
# norm_sol = [norm(exp(A * tk) * real.(x0)) for tk in t]

# plot(t, norm_sol, label="Norm of solution to dx/dt = Ax, x(0)=x0", xlabel="t", ylabel="||exp(A*t) x0||")


### END SOLUTION

In [ ]:
# Do your own checking here

x0 = real.(F.vectors[:,4])

In [ ]:
#Grader Cell. 1 point
### BEGIN HIDDEN TESTS
# It's still correct if x0 is a linear combination of the two real eigenvectors
@assert rank([x0 F.vectors[:, [1,4]]],  atol=1e-3) == 2
### END HIDDEN TESTS

# Problem 6: Using Code to Solve  $\dot{x}(t) = A \cdot x(t), x(0) = x_0 \in \mathbb{R}^n$ via Laplace Transforms

Or, how to compute $e^{At} \cdot x_0$ in closed form. 

### Laplace Transform Setup

The Laplace transform is a linear operation and extends naturally to vectors and matrices.  

If we define
$$
X(s) = \mathcal{L}\{ x(t) \} 
= \begin{bmatrix} 
\mathcal{L}\{ x_1(t) \}  \\ 
\vdots \\ 
\mathcal{L}\{ x_n(t) \} 
\end{bmatrix},
$$
then
$$
\begin{aligned}
\mathcal{L}\{ \dot{x}(t) \} &= s X(s) - x(0^-) \\
\mathcal{L}\{ A x(t) \} &= A X(s).
\end{aligned}
$$

### Derivation

$$
\begin{aligned}
s X(s) - x(0^-) &= A X(s) \\
\Updownarrow \\
(sI_n - A) X(s) &= x(0^-) \\
\Updownarrow \\
X(s) &= (sI_n - A)^{-1} \, x(0^-).
\end{aligned}
$$

Taking the inverse Laplace transform gives
$$
x(t) = \mathcal{L}^{-1}\{ X(s) \} 
= \mathcal{L}^{-1}\{ (sI_n - A)^{-1} x(0^-) \} 
= e^{At} \, x_0.
$$

### Practical Note

Doing this by hand is tedious even for $2 \times 2$ systems. The textbook shows examples with **SymPy**, but beyond small systems SymPy struggles. That is why we provide you with a custom function:

```julia
F = Laplace(A, x0, s, t)   # in HW08_LaplaceTransformUtilities.jl
```
This function returns:
  - `F.X_s` = X(s) is the vector of Laplace transforms
  - `F.x_t` = x(t) = exp(At) x0 is the closed-form time-domain solution in three forms:
    1. A vector of time functions
    2. Each component separately as a function of $t$
    3. A Julia function to evaluate $x(t)$ at a specific time or broadcast over a vector of times

You will find the first two forms to be useful in your courses that use Laplace transforms, such as EECS 216, ME 360, EECS 460, and ME 461.

### Unit Step Function

SymPy uses $\theta(t)$ to denote the unit step function: $$\theta(t):= \begin{cases} 1.0 & t > 0 \\\\ 0.0 & t < 0 \\\\ \text{don't care} & t = 0\end{cases} $$

In this course we’ve been using $u_{\text{stp}}(t)$. Mathematicians call this the **Heaviside function**.

#### Example: Mass-Spring-Damper System

We consider $\dot{x} = A x$, with $x_0 \in \mathbb{R}^2$.

```julia
# Parameters
m = 5.0   # Mass (kg)
k = 2.0   # Spring constant (N/m)
b = 3.0   # Damping coefficient (Ns/m)

A = [0 1; -k/m -b/m]
```

---

### Instructions

1. Compute $X(s)$, the Laplace transform of the state vector
2. Compute its inverse Laplace transform to obtain the closed-form solution of the ODE
3. Build functions from the solution for evaluation and plotting

**Note:** `SymPy` and `PyCall` allow Julia to use Python’s symbolic math library. The syntax looks like Python and is not standard Julia.


In [ ]:
using SymPy, PyCall, LinearAlgebra

# Import Python's SymPy for using its inverse Laplace transform functionality
sympy = pyimport("sympy")

# Define 's' as a complex symbol and 't' as a real symbol
s = symbols("s")
t = symbols("t", real=True)

#Include custom functions developed by JWG
include("HW08_LaplaceTransformUtilities.jl")

# Parameters for the mass spring damper system
m = 5.0   # Mass in kg
k = 2.0   # Spring constant in N/m
b = 3.0   # Damping coefficient in Ns/m

# Define the matrix A and initial condition vector x0 for the system dx/dt = Ax, x(0) = x0
A = [0 1; -k/m -b/m]
x0 = [1, 0]          # Change the initial conditions as desired

# Use a custom function to compute the Laplace transform of the solution 
# vector X_s, and its inverse Laplace transform, x_t
F = Laplace(A, x0, s, t)

# Vector of Laplace Transforms
X_s = F.X_s

println("Laplace Transform\n")
display(X_s)

# Display each component of X_s
println("Print each component in a more readable form: \n")
for (i, comp) in enumerate(X_s)
    println("Component $i of X_s: ", simplify(comp))
end

We next extract and display the time-domain solution.

In [ ]:
# Vector of time fuctions
x_t = F.x_t
println("\nx(t): Inverse Laplace Transform as a Vector of Time Functions\n")
display(x_t)

# Display the inverse Laplace transform of each individual component
println("\n x(t): Inverse Laplace Transform one Component at a Time: \n")
for (idx, solution) in enumerate(x_t)
    println("Component $idx: $solution \n")
    #display(cleanUpExpression(solution))
end

In your engineering courses, it may be useful to copy and paste the "text" after each i-th-Component to create a function as shown below:

In [ ]:
import SymPy: Heaviside

# Redefine the Heaviside function
Heaviside(t) = t >= 0 ? 1.0 : 0.0 # same as our unit step function in lecture

# x1(t) = "copy and paste from the output of the above cell"
x1(t) =  (0.538815906080325*sin(0.556776436283002*t) + 1.0*cos(0.556776436283002*t))*exp(-0.3*t)*Heaviside(t)  

# evaluate on a vector of points
tValues = 0:0.1:25

x1.(tValues) # use broadcasting

As an alternative, we have created a custom function that transforms the entire vector of time functions ```x_t``` into a Julia function as illustrated next.

**Notes:** If all you need is the solution evaluated at a vector of points, then solving the ODE via `DifferntialEqualtions.jl` is more efficient. The function ``mySolution`` is useful if you want to make a plot to go along with your closed-form solution, x_t 

In [ ]:
#Include custom functions developped by JWG
include("HW08_LaplaceTransformUtilities.jl")

In [ ]:
# Example of transforming x_t into a Julia function named mySolution
# You can name the function anything you want.
# Here, we called it mySolution
#
mySolution = mySolutionFactory(x_t)

# Example usage: Evaluate the solution at a range of times
tValues = 0:0.1:25

println("Each row is a solution component evaluated on tValues")
solution_matrix = mySolution(tValues)
display(solution_matrix)

using Plots, LaTeXStrings
x_1 = solution_matrix[1, :]
plot(tValues, x_1, lw=2, color=:blue, xlabel="t (sec)", ylabel=L"x_1(t)", legend=false)

## P.6A: For the linear ODE, $ \dot{x} = A x,  x(0)=x_0,$ compute the laplace transformation of $e^{At} \cdot x_0$ 

Given: 

```julia
A = [-0.283447   2.32283    0.302781   -2.85082
  0.163857  -0.642933  -0.228616   -0.596354
 -1.01479    1.38138    0.0542474  -0.302595
 -0.345692   0.661838   0.769124   -1.12787]

F = eigen(A)

x0 = real(F.vectors[:,2])
```

You are to find $X(s) = (s I - A)^{-1} \cdot x0$. Follow the process used for the mass-spring-damper ODE.

In [ ]:
# X_s = ??


### BEGIN SOLUTION 
using SymPy, PyCall, LinearAlgebra

# Import Python's SymPy for using its inverse Laplace transform functionality
sympy = pyimport("sympy")

# Define 's' as a complex symbol and 't' as a real symbol
s = symbols("s")
t = symbols("t", real=True)

A = [-1.20765   -0.576936  -4.65319  -1.30087
 -5.11041    0.866501  -9.48172  -4.64823
  0.841043  -0.389652  -1.39981   0.214517
 -1.12432    1.35583    4.62568  -0.259039]


F = eigen(A)
x0 = real(F.vectors[:,2])

#Include custom functions developped by JWG
include("HW08_LaplaceTransformUtilities.jl")


# Compute the Laplace transform of the solution vector, X_s
# Compute the Laplace transform of the solution vector, X_s, and its inverse Laplace transform, x_t
F = Laplace(A, x0, s, t)

X_s = F.X_s
println("Laplace Transform")
display(X_s)
### END SOLUTION

println("\nFirst component of the Laplace transform")
println(X_s[1])

> ### Friendly Check
> 
> After a bit of rounding,
> 
> ```julia
> X_s[1] = (-0.7553*s - 0.7553)/(1.0*s^2 + 2.0000*s + 2.0000)
> ```
> 
> The four denominators are the same.

In [ ]:
#Grader Cell. 1 point
### BEGIN HIDDEN TESTS
num = sympy.numer(X_s[1])
# Convert numerator to polynomial and get its coefficients
num_poly = sympy.Poly(num, s)
coeffs = num_poly.coeffs()
#print(coeffs)
Answer = Sym[-0.755346574801414, -0.755347040065316]
@assert(sum(abs.(coeffs - Answer)) < 1e-3)
### END HIDDEN TESTS

## P.6B: Compute a closed-form solution to $ \dot{x} = A x,~~x(0)=x_0$

Given:
```julia
A = [-1.20765   -0.576936  -4.65319  -1.30087
 -5.11041    0.866501  -9.48172  -4.64823
  0.841043  -0.389652  -1.39981   0.214517
 -1.12432    1.35583    4.62568  -0.259039]

F = eigen(A)

x0 = real(F.vectors[:,2])
```

You are to find x_t. If you have computed x_t in the previous problem, then this part can be one line.

In [ ]:

# x_t =  ??

### BEGIN SOLUTION 

using SymPy, PyCall, LinearAlgebra
# Import Python's SymPy for using its inverse Laplace transform functionality
sympy = pyimport("sympy")

#Include custom functions developped by JWG
include("HW08_LaplaceTransformUtilities.jl")


# Define 's' as a complex symbol and 't' as a real symbol
s = symbols("s")
t = symbols("t", real=True)

A = [-1.20765   -0.576936  -4.65319  -1.30087
 -5.11041    0.866501  -9.48172  -4.64823
  0.841043  -0.389652  -1.39981   0.214517
 -1.12432    1.35583    4.62568  -0.259039]


F = eigen(A)
display(F.values)

x0 = real.(F.vectors[:,2]) 

F = Laplace(A, x0, s, t)

x_t = F.x_t


### END SOLUTION

# Assumes you have called your answer x_t
x_t = simplify(x_t)
# Display the inverse Laplace transform of each individual component
println("\n x(t): Inverse Laplace Transform one Component at a Time: \n")
for (idx, solution) in enumerate(x_t)
    println("Component $idx: $solution \n")
    #display(solution)
end

> ### Friendly Check
> 
> After a bit of rounding
> 
> ```julia
> Component 1: -0.7553*exp(-1.0000*t)*cos(1.0000*t)*Heaviside(t) 
> 
> Component 4: -(0.1236*sin(1.0000*t) + 0.2977*cos(1.0000*t))*exp(-1.0000*t)*Heaviside(t) 
> 
> # Note: 0.297677875141958 rounds to 0.2977
> ```


In [ ]:
#Grader Cell. 1 point
### BEGIN HIDDEN TESTS
mySolutionProb6B = mySolutionFactory(x_t)
print(mySolutionProb6B(1))
Answer = [-0.1501369220494081; 0.03024350815079123; -0.021840070087866467; -0.09743466868040462]
@assert(norm(mySolutionProb6B(1) -Answer) < 1e-3)
### END HIDDEN TESTS

# Problem 7: Return of the Mass-Spring-Damper

For the Linear ODE, $ \dot{x} = A x$, compute the matrix exponential $e^{At}$ in closed form.

<br>
<p align="center">
  <img src="./data/HW08MassSpringDamper.png", width="40%">
</p>
<br>

$$
\left[ \begin{array}{c} \dot{x}_1  \\\\ \dot{x}_2 \end{array} \right] = \left[ \begin{array}{cc} 0 & 1\\\\- \frac{k}{m} & -\frac{b}{m} \end{array}\right] \left[ \begin{array}{c} x_1  \\\ x_2 \end{array}\right],
$$

Given:
```julia
# Parameters
m = 5.0   # Mass in kg
k = 2.0   # Spring constant in N/m
b = 3.0   # Damping coefficient in Ns/m

A = [0 1; -k/m -b/m]
```

You are to find $e^{At}$ in closed form. Call you answer `expAt`.

**Hint:** With the tools you have been given, you can compute it column by column.

In [ ]:

# expAt =  ?? $ should be a 2 x 2 matrix of time functions


### BEGIN SOLUTION 

using SymPy, PyCall, LinearAlgebra
# Import Python's SymPy for using its inverse Laplace transform functionality
sympy = pyimport("sympy")

#Include custom functions developped by JWG
include("HW08_LaplaceTransformUtilities.jl")


# Define 's' as a complex symbol and 't' as a real symbol
s = symbols("s")
t = symbols("t", real=True)

# A matrix
m = 5.0   # Mass in kg
k = 2.0   # Spring constant in N/m
b = 3.0   # Damping coefficient in Ns/m
A = [0 1; -k/m -b/m]

# Define the identity matrix and initialize expAt
n, m = size(A)
myI = I(n)

expAt = Sym[Sym("f_$(i)_$(j)(t)") for i in 1:n, j in 1:n]
#display(expAt)


for k = 1:n
    x0 = myI[:,k]
    F = Laplace(A, x0, s, t)
    expAt[:, k] = F.x_t
end


### END SOLUTION
println("\n\n expAt[1,2] =", expAt[1,2], "\n\n")
display(expAt)

> ## Friendly Check: 
> 
> $e^{At}\big|_{t=0} = I$
>
> ```julia
> expAt[1,2] = 1.796*exp(-0.3*t)*sin(0.557*t)*Heaviside(t) # After some rounding 
> ```



In [ ]:
#Grader Cell. 1 point
### BEGIN HIDDEN TESTS
mySolutionProb7 = mySolutionFactory(expAt)
#print(mySolutionProb7(1))
Answer = [0.8398667301464158; -0.2812526097708068; 0.7031315244270181; 0.4179878154902049]
@assert(norm(mySolutionProb7(1) - Answer) < 1e-3)
### END HIDDEN TESTS

<p align="center">
  <img src="./data/HW08FuturisticClassRoomMarch2025.png", width="70%">
</p>


# Extra: Improved Means to Compute exp(At)

In [ ]:
#=
Main script to solve a linear ODE system dx/dt = Ax, x(0) = x0
using the real basis eigenvalue method implemented in the utility file.

You have a choice of a mass-spring-damper system or 
a randomly selected 12 dimensional model
=#

example = "massSpringDamper" 

example = "alternative"
Nsize = 8 # size of alternative random example

using SymPy       # For symbolic math
using PyCall      # To interact with Python's SymPy
using LinearAlgebra # For matrix operations etc.

# Import Python's SymPy via PyCall (optional if not used directly elsewhere)
# sympy = pyimport("sympy")

# Define symbolic variable for time 't'
# Ensure real=True for exp, sin, cos to work correctly in symbolic results
t = SymPy.symbols("t", real=True)
# Laplace variable 's' is NOT needed for this method

# 
include("RealBasisMatrixExponentialUtilities.jl")


# --- Problem Definition ---
# Parameters for the mass-spring-damper system (Example)
m = 5.0   # Mass (kg)
k = 2.0   # Spring constant (N/m)
b = 3.0   # Damping coefficient (Ns/m)

if example == "massSpringDamper"
    # Define the system matrix A for dx/dt = Ax
    # (State variables: x = [position; velocity])
    A = [0.0 1.0; -k/m -b/m]

    # Define the initial condition vector x(0) = x0
    x0 = [1.0, 0.0] # Example: Initial position = 1, Initial velocity = 0
else
    A = randn(Nsize,Nsize)
    x0 = randn(Nsize)
end

println("System Matrix A:")
display(A)
println("\nInitial Condition x0:")
display(x0)

println("\nEigenvalues and Eigenvectors")
F = eigen(A)
display(F)

# --- Solve the ODE System ---
# Call the solver function (note: only 't' is passed, not 's')
solution = solve_ode_real_basis(A, x0, t)

# --- Display Results ---
# Check if the solution was successful (solver returns NamedTuple or nothing)
if !isnothing(solution)
    # Unpack the result - ONLY 'x_t' is returned by this solver
    x_t = solution.x_t

    # --- Display Time Domain Solution x(t) ---
    println("\n--- Time Domain Solution x(t) ---")
    println("Vector x(t):")
    display(x_t) # Displays the symbolic vector solution

    println("\nComponents of x(t): \n")
    for (idx, sol_comp) in enumerate(x_t)
        println("Component $idx: ", sol_comp, "\n")
    end

else
    # Message if the solver function returned 'nothing'
    println("\nFailed to solve the ODE system using the real basis method.")
end

<p align="center" style="font-size:48px;"><strong>The End! </strong></p>